# 0. Initialisation

In [2]:
import pandas as pd
import numpy as np 
from helpers2 import S3Connection

In [3]:
s3 = S3Connection(bucket_name="clichere/diffusion")

Connection successful


# 1. Ouverture et Nettoyage des fichiers DPE

## 1.1. DPE après juillet 2021

In [71]:
path_logements_existants = "DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
DPE_logements_existants = s3.read_csv_from_s3(path_logements_existants)

In [72]:
path_logements_neufs = "DPE/DPE_ADEME/dpe-v2-logements-neufs.csv"
DPE_logements_neufs = s3.read_csv_from_s3(path_logements_neufs)

In [73]:
print(DPE_logements_existants.shape)
print(DPE_logements_neufs.shape)

(4537525, 33)
(537952, 30)


In [74]:
# Nombre total d'observations
total_obs1 = len(DPE_logements_existants)
total_obs2 = len(DPE_logements_neufs)

# Nombre de doublons (hors première occurrence)
nb_doublons1 = DPE_logements_existants.duplicated().sum()
nb_doublons2 = DPE_logements_neufs.duplicated().sum()

# Calcul du ratio
ratio_doublons1 = nb_doublons1 / total_obs1
ratio_doublons2 = nb_doublons2 / total_obs2

print(ratio_doublons1)
print(ratio_doublons2)

0.05455352863069625
0.19296703051573375


Environ 5,5% des observations des logements existants sont des doublons mais 19,3% des logements neufs le sont.

In [75]:
DPE_logements_existants = DPE_logements_existants.drop_duplicates()
DPE_logements_neufs = DPE_logements_neufs.drop_duplicates()

In [76]:
set(DPE_logements_existants.columns) - set(DPE_logements_neufs.columns)

{'Année_construction', 'Méthode_application_DPE', 'Période_construction'}

In [77]:
print(DPE_logements_existants.columns)

Index(['Date_réception_DPE', 'Date_établissement_DPE', 'Modèle_DPE',
       'Date_fin_validité_DPE', 'Version_DPE', 'Méthode_application_DPE',
       'Etiquette_DPE', 'Etiquette_GES', 'Année_construction', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_brute',
       'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)',
       'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN',
       'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'Code_postal_(brut)',
       'N°_étage_appartement', 'Nom_résidence', 'Cage_d'escalier',
       'Complément_d'adresse_logement', 'Statut_géocodage',
       'Nom__commune_(Brut)', 'N°_département_(BAN)', 'N°_région_(BAN)',
       'Complément_d'adresse_bâtiment'],
      dtype='object')


In [ ]:
print(DPE_logements_existants["Modèle_DPE"].value_counts(dropna=False))
print(DPE_logements_neufs["Modèle_DPE"].value_counts(dropna=False))

print(DPE_logements_existants["Période_construction"].value_counts(dropna=False))

Modèle_DPE
DPE 3CL 2021 méthode logement    4289987
Name: count, dtype: int64
Modèle_DPE
DPE NEUF logement : RT2012    419726
DPE NEUF logement : RE2020     14419
Name: count, dtype: int64
Période_construction
1948-1974     1345477
avant 1948    1279994
1989-2000      390692
2006-2012      321454
1978-1982      229330
1983-1988      198796
2013-2021      181024
1975-1977      173102
2001-2005      140513
après 2021      29605
Name: count, dtype: int64


Il y a trois variables en plus dans le fichier logements : Année_construction, Période_construction et Méthode application DPE. Dans le cadre de notre analyse, il nous semble utile de garder l'année de construction et la période de construction, la deuxième variable étant plus complète. Nous pouvons enlever la variable "Méthode_application_DPE".
Par ailleurs, nous allons sélectionner seulement les variables qui peuvent nous intéresser pour l'analyse. 

In [119]:
DPE_logements_neufs["Année_construction"] = DPE_logements_neufs["Date_établissement_DPE"].astype(str).str[:4]
DPE_logements_neufs["Période_construction"] = "Après 2021"
DPE_logements_existants_clean = DPE_logements_existants.drop(columns=[
       "Méthode_application_DPE","Adresse_brute",
       "Nom__commune_(BAN)", "N°_voie_(BAN)",
       "Identifiant__BAN", "Code_postal_(BAN)",
       "Nom__rue_(BAN)","Code_postal_(brut)",
       "Cage_d'escalier",
       "Complément_d'adresse_logement",
       "Nom__commune_(Brut)", "N°_département_(BAN)",
       "Complément_d'adresse_bâtiment", "Nom_résidence", "Statut_géocodage", "Date_réception_DPE", "Date_fin_validité_DPE"])
DPE_logements_neufs_clean = DPE_logements_neufs.drop(columns=["Adresse_brute",
       "Nom__commune_(BAN)", "N°_voie_(BAN)",
       "Identifiant__BAN", "Code_postal_(BAN)",
       "Nom__rue_(BAN)","Code_postal_(brut)",
       "Cage_d'escalier",
       "Complément_d'adresse_logement",
       "Nom__commune_(Brut)", "N°_département_(BAN)",
       "Complément_d'adresse_bâtiment", "Nom_résidence", "Statut_géocodage", "Date_réception_DPE", "Date_fin_validité_DPE"])

In [122]:
print(DPE_logements_existants_clean.shape)
print(DPE_logements_neufs_clean.shape)

(4289987, 16)
(434145, 16)


In [123]:
DPE_logements = pd.concat([DPE_logements_existants_clean, DPE_logements_neufs_clean], ignore_index=True)

In [124]:
DPE_logements['Année_construction'] = pd.to_numeric(DPE_logements['Année_construction'], errors='coerce')

In [125]:
print(DPE_logements.shape)
DPE_logements.sample(20)

(4724132, 16)


,Date_établissement_DPE,Modèle_DPE,Version_DPE,Etiquette_DPE,Etiquette_GES,Année_construction,Type_bâtiment,Période_construction,Surface_habitable_logement,Code_INSEE_(BAN),Adresse_(BAN),Score_BAN,Coordonnée_cartographique_X_(BAN),Coordonnée_cartographique_Y_(BAN),N°_étage_appartement,N°_région_(BAN)
1326916,2021-07-07,DPE 3CL 2021 méthode logement,1.0,F,F,1947.0,appartement,avant 1948,45.9,69384,22 Rue Jacquard 69004 Lyon,0.68,8.420034e+05,6521222.29,0.0,84.0
3770330,2022-07-04,DPE 3CL 2021 méthode logement,2.1,E,D,NaN,maison,avant 1948,168.5,77099,23 Rue de la République 77570 Château-Landon,0.54,6.776452e+05,6783072.65,0.0,11.0
1199068,2023-02-16,DPE 3CL 2021 méthode logement,2.2,D,B,1980.0,maison,1978-1982,87.9,03102,369 Rue du Docteur Rouis 03290 Dompierre-sur-B...,0.52,7.515612e+05,6602772.23,0.0,84.0
3011950,2023-04-06,DPE 3CL 2021 méthode logement,2.3,E,E,1976.0,maison,1975-1977,99.2,79049,22 Rue de la Cabane 79300 Bressuire,0.96,4.342794e+05,6644502.39,NaN,75.0
803922,2021-10-30,DPE 3CL 2021 méthode logement,1.1,D,B,1948.0,appartement,1948-1974,27.3,22067,65 Rue de la Madeleine 22200 Grâces,0.52,2.454598e+05,6846425.74,0.0,53.0
1351325,2022-02-16,DPE 3CL 2021 méthode logement,2.1,C,A,1947.0,appartement,avant 1948,76.0,09194,10 Rue Delcassé 09500 Mirepoix,0.52,6.082076e+05,6221867.40,0.0,76.0
4317921,2022-07-12,DPE NEUF logement : RT2012,2.1,A,A,2022.0,maison,Après 2021,134.0,35013,1 Le Bois Montant 35600 Bains-sur-Oust,0.80,3.207786e+05,6745015.71,NaN,53.0
2605434,2022-10-23,DPE 3CL 2021 méthode logement,2.2,A,A,2011.0,appartement,2006-2012,45.6,94021,6 Rue du Clos Saint-michel 94550 Chevilly-Larue,0.56,6.523405e+05,6852675.40,0.0,11.0
1289236,2022-08-26,DPE 3CL 2021 méthode logement,2.1,E,B,1974.0,maison,1948-1974,73.0,83073,2 Rue Eugene Garnier 83340 Le Luc,0.87,9.685543e+05,6261133.43,NaN,93.0
2773639,2023-02-03,DPE 3CL 2021 méthode logement,2.2,C,A,NaN,maison,1989-2000,101.3,33032,15 Rue de Verdun 33530 Bassens,0.54,4.229295e+05,6429330.58,0.0,75.0


In [126]:
doublons3 = DPE_logements.duplicated().sum()
total_obs3 = len(DPE_logements)
ratio_doublons3 = doublons3/total_obs3
print(ratio_doublons3)

0.13743561780238148


In [127]:
DPE_logements = DPE_logements.drop_duplicates()

In [128]:
outpath1 = 'DPE/DPE_après_2021.parquet'

s3.from_pandas_to_parquet_store_in_s3(DPE_logements, outpath1)

Fichier Parquet écrit avec succès dans DPE/DPE_après_2021.parquet


# 2. Ouverture et nettoyage des fichiers valeurs foncières

## 2.1. Téléchargement et fusion des bases

In [6]:
path_vf = "Valeursfoncières/full.csv"
vf_new = s3.read_csv_from_s3(path_vf)

In [7]:
vf_new.sample(20)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
420477,2024-142955,2024-01-04,1,Vente en l'état futur d'achèvement,290000.0,NaN,NaN,RUE DE CANDIA,0200,20090.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.748817,41.937222
49204,2024-15059,2024-04-23,1,Vente,28770.0,NaN,NaN,LES SIGNIERES,B092,4330.0,...,NaN,NaN,NaN,L,landes,VAGUE,Terrain vague,3222.0,6.266320,43.948698
1189451,2024-417008,2024-06-21,1,Vente,245000.0,114.0,NaN,RUE DE L ETANG,0391,76190.0,...,Dépendance,NaN,0.0,S,sols,NaN,NaN,691.0,0.769587,49.614977
1502712,2024-527678,2024-03-21,1,Vente,120000.0,9.0,NaN,RUE DE LA FORGE,0452,95320.0,...,Appartement,38.0,2.0,NaN,NaN,NaN,NaN,NaN,2.246431,49.016828
603126,2024-209785,2024-06-04,1,Vente,2000.0,NaN,NaN,LA BELLE CAVE,B004,37220.0,...,NaN,NaN,NaN,T,terres,NaN,NaN,2440.0,0.391470,47.086061
249947,2024-88030,2024-02-29,1,Vente,508650.0,NaN,NaN,RUE DE LA GRANDE ROMADE,0095,17460.0,...,NaN,NaN,NaN,AG,terrains d'agrément,NaN,NaN,579.0,-0.742205,45.688224
1406796,2024-492360,2024-06-24,1,Vente,48000.0,NaN,NaN,NOYER THIBAULT,B076,89400.0,...,NaN,NaN,NaN,BT,taillis simples,NaN,NaN,2650.0,3.550640,47.928814
1247546,2024-437762,2024-05-17,1,Vente,66358.0,NaN,NaN,LA CRETINIERE,B023,79240.0,...,NaN,NaN,NaN,P,prés,NaN,NaN,1458.0,-0.579394,46.618274
590942,2024-205516,2024-01-11,1,Vente,27000.0,NaN,NaN,RUE DE VILLENEUVE,0949,37380.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.775860,47.494446
479576,2024-164733,2024-01-18,1,Vente,180700.0,20.0,NaN,IMP DU MASCARAS,0040,32140.0,...,Maison,67.0,3.0,P,prés,NaN,NaN,861.0,0.597714,43.353023


In [4]:
dtype_spec = {"Code commune": str}

path_valeursfoncieres_2022 = "Valeursfoncières/valeursfoncieres-2022.txt"
vf2022 = s3.read_txt_from_s3(path_valeursfoncieres_2022, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2021 = "Valeursfoncières/valeursfoncieres-2021.txt"
vf2021 = s3.read_txt_from_s3(path_valeursfoncieres_2021, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2020 = "Valeursfoncières/valeursfoncieres-2020.txt"
vf2020 = s3.read_txt_from_s3(path_valeursfoncieres_2020, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2019 = "Valeursfoncières/valeursfoncieres-2019.txt"
vf2019 = s3.read_txt_from_s3(path_valeursfoncieres_2019, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2018 = "Valeursfoncières/valeursfoncieres-2018.txt"
vf2018 = s3.read_txt_from_s3(path_valeursfoncieres_2018, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2017 = "Valeursfoncières/valeursfoncieres-2017.txt"
vf2017 = s3.read_txt_from_s3(path_valeursfoncieres_2017, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2016 = "Valeursfoncières/valeursfoncieres-2016.txt"
vf2016 = s3.read_txt_from_s3(path_valeursfoncieres_2016, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2015 = "Valeursfoncières/valeursfoncieres-2015.txt"
vf2015 = s3.read_txt_from_s3(path_valeursfoncieres_2015, sep="|", dtype_spec=dtype_spec)

path_valeursfoncieres_2014 = "Valeursfoncières/valeursfoncieres-2014.txt"
vf2014 = s3.read_txt_from_s3(path_valeursfoncieres_2014, sep="|", dtype_spec=dtype_spec)


In [5]:
print(vf2014.shape)
print(vf2015.shape)
print(vf2016.shape)
print(vf2017.shape)
print(vf2018.shape)
print(vf2019.shape)
print(vf2020.shape)
print(vf2021.shape)
print(vf2022.shape)

(2516688, 43)
(2749830, 43)
(2936524, 43)
(3383622, 43)
(3335292, 43)
(3625903, 43)
(3514698, 43)
(4649209, 43)
(3803885, 43)


In [131]:
vf = pd.concat([vf2022, vf2021, vf2020, vf2019, vf2018, vf2017, vf2016, vf2015, vf2014], ignore_index=True)


In [132]:
print(vf.columns)

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code service CH'],
      dtype='object')


In [133]:
print(vf.dtypes)

Identifiant de document       float64
Reference document            float64
1 Articles CGI                float64
2 Articles CGI                float64
3 Articles CGI                float64
4 Articles CGI                float64
5 Articles CGI                float64
No disposition                  int64
Date mutation                  object
Nature mutation                object
Valeur fonciere                object
No voie                       float64
B/T/Q                          object
Type de voie                   object
Code voie                      object
Voie                           object
Code postal                   float64
Commune                        object
Code departement               object
Code commune                   object
Prefixe de section            float64
Section                        object
No plan                         int64
No Volume                      object
1er lot                        object
Surface Carrez du 1er lot      object
2eme lot    

In [134]:
print(vf["Nature mutation"].value_counts(dropna=False))
print(vf["1er lot"].value_counts(dropna=False))

Nature mutation
Vente                                 27775517
Vente en l'état futur d'achèvement     2085142
Echange                                 405566
Adjudication                            111197
Vente terrain à bâtir                    96519
Expropriation                            41710
Name: count, dtype: int64
1er lot
NaN      21035867
1          326170
2          268312
3          210252
10         179842
           ...   
33K             1
20805           1
7372            1
7644            1
7700            1
Name: count, Length: 22126, dtype: int64


In [ ]:
vf_clean = vf[['Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Voie', 'Code postal',
       'Commune', 'Code commune', '1er lot',
       'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot',
       'Surface Carrez du 3eme lot',
       'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot',
       'Type local','Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain']]

In [137]:
vf_clean[vf_clean["Type local"] == "Appartement"].sample(20)

,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Code commune,...,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
6938854,29/07/2021,Vente,"276610,00",43.0,NaN,AV,PAUL DELORME,69580.0,SATHONAY-CAMP,292,...,NaN,NaN,NaN,NaN,Appartement,66.0,3.0,NaN,NaN,NaN
22415174,22/03/2016,Vente,"261000,00",252.0,NaN,AV,DE LA LANTERNE,6200.0,NICE,88,...,NaN,NaN,NaN,NaN,Appartement,80.0,3.0,NaN,NaN,NaN
4632904,25/10/2021,Vente,"79000,00",3.0,NaN,SQ,DU CREUX D ENFER,21000.0,DIJON,231,...,NaN,NaN,NaN,NaN,Appartement,25.0,1.0,NaN,NaN,NaN
26252910,07/02/2015,Vente,"255000,00",14.0,NaN,RUE,DU MARCHE,36000.0,CHATEAUROUX,44,...,NaN,NaN,NaN,NaN,Appartement,26.0,1.0,S,NaN,279.0
16304596,11/12/2018,Vente en l'état futur d'achèvement,"230000,00",5.0,NaN,RUE,DU COLLEGE,25800.0,VALDAHON,578,...,NaN,NaN,NaN,NaN,Appartement,83.0,3.0,NaN,NaN,NaN
19265936,02/03/2017,Vente,"183000,00",125.0,NaN,AV,DU 24 AVRIL 1915,13012.0,MARSEILLE 12EME,212,...,NaN,NaN,NaN,NaN,Appartement,84.0,5.0,NaN,NaN,NaN
10780551,18/08/2020,Vente,"122100,00",3.0,NaN,RUE,DES SEPT PRIMADIE,66000.0,PERPIGNAN,136,...,NaN,NaN,NaN,NaN,Appartement,90.0,4.0,S,NaN,177.0
22061850,11/10/2017,Vente,"255000,00",4.0,NaN,RUE,DU CHEMIN VERT,92130.0,ISSY-LES-MOULINEAUX,40,...,NaN,NaN,NaN,NaN,Appartement,45.0,2.0,NaN,NaN,NaN
9063579,12/10/2020,Vente,"230000,00",57.0,NaN,RUE,DE LONGVIC,21000.0,DIJON,231,...,"82,04",NaN,NaN,NaN,Appartement,80.0,3.0,NaN,NaN,NaN
30512937,18/11/2014,Vente,"0,00",5.0,NaN,RUE,JACQUES PREVERT,75020.0,PARIS 20,120,...,NaN,NaN,NaN,NaN,Appartement,72.0,4.0,S,NaN,2656.0


In [19]:
outpath2 = 'Valeursfoncières/vf.parquet'

s3.from_pandas_to_parquet_store_in_s3(vf_clean, outpath2)

Fichier Parquet écrit avec succès dans Valeursfoncières/vf.parquet


### Normalisation des adresses

In [ ]:
# Convertir explicitement toutes les colonnes en chaînes
#vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Type de voie'].fillna('').astype(str) + " " + \
 #               vf['Voie'].fillna('').astype(str) + ", " + \
 #               vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
 #               vf['Commune'].fillna('').astype(str)
#print(vf.describe)

In [ ]:
#vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides
#print(vf2022[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Adresse']].head())

In [ ]:
#vf2022['Adresse_Normalisee'] = vf2022['Adresse'].apply(s3.normalize_address)

#print(vf2022[['Adresse', 'Adresse_Normalisee']].head())

TypeError: s3_connection.normalize_address() takes 1 positional argument but 2 were given